TRABALHO BD

importar um registo e guardar uma string locus
Ir buscar o ficheiro GenBank pelo webscrapping

A partir daqui ir buscar locus, id, definition, accession... pelas expressões regulares

Colocar as infos na BD

Escolha de sequências
-> escolhemos 1 vírus (HIV) e uma bactéria (H. pylori)
-> motivos das escolhas: sequências pequenas (teoricamente) e mais fácil de analisar

WEBSCRAPPING

In [143]:
import time
query = input("Defina o que pretende pesquisar [bacteria/yeast/...]: ")
def url_get(item):
    url_list = []
    url = "https://www.ncbi.nlm.nih.gov/gene/?term={}".format(query)
    url_list.append(url)
    return url_list
    
url_get(0)

['https://www.ncbi.nlm.nih.gov/gene/?term=human']

In [144]:
import requests, sys, json
content = []
for url in url_get(0):
    r = requests.get(url)
    content.append(r.content)
# print(content)

In [145]:
import requests
from bs4 import BeautifulSoup

# Parsing the HTML
for c in content:
    soup = BeautifulSoup(r.content, 'html.parser')
    res = soup.get_text()
# print(res)

In [146]:
import re
existe = re.findall(r'ID:\s*\d*(?=\D)', res, re.DOTALL)
id_genes = ','.join(existe)
id_g = re.sub(r'ID:\n*\s+', '', id_genes)
id_gene = id_g.split(',')
id_gene


['9606',
 '348',
 '1636',
 '1588',
 '54658',
 '10',
 '1071',
 '3106',
 '116519',
 '8091',
 '10599',
 '28',
 '3159',
 '1021',
 '3990',
 '2120',
 '335',
 '5071',
 '3949',
 '3992',
 '255738']

In [147]:
number_of_genes = int(input("Número de genes a obter (máx: 20): "))
print(number_of_genes)
genes = id_gene[0:number_of_genes]
print(genes)

10
['9606', '348', '1636', '1588', '54658', '10', '1071', '3106', '116519', '8091']


In [ ]:
#BUSCAR LINKS DOS GENES

In [148]:
import time
def url_get_id(item):
    url_id = []
    for id in genes:
        url = f"https://www.ncbi.nlm.nih.gov/nuccore/{id}"
        url_id.append(url)
    return url_id
url_get_id(genes)

['https://www.ncbi.nlm.nih.gov/nuccore/9606',
 'https://www.ncbi.nlm.nih.gov/nuccore/348',
 'https://www.ncbi.nlm.nih.gov/nuccore/1636',
 'https://www.ncbi.nlm.nih.gov/nuccore/1588',
 'https://www.ncbi.nlm.nih.gov/nuccore/54658',
 'https://www.ncbi.nlm.nih.gov/nuccore/10',
 'https://www.ncbi.nlm.nih.gov/nuccore/1071',
 'https://www.ncbi.nlm.nih.gov/nuccore/3106',
 'https://www.ncbi.nlm.nih.gov/nuccore/116519',
 'https://www.ncbi.nlm.nih.gov/nuccore/8091']

In [149]:
import requests
id_content = []
for url in url_get_id(genes):
    r = requests.get(url)
    id_content.append(r.content)
# id_content

In [150]:
import requests
from bs4 import BeautifulSoup

# Making a GET request
r = requests.get('https://www.ncbi.nlm.nih.gov/nuccore/{}'.format(genes))
# Parsing the HTML
locus = ""
for c in id_content:
	soup = BeautifulSoup(c, 'html.parser')

# Procurar um tag meta com um determinado atributo

	lines = soup.find_all('meta', {'name':"ncbi_uidlist"} )

	id = ""
	url = ""
	for line in lines:
		# print(line)
		# if 'name' in line.attrs:
		# 	print(line.attrs['name'])
		if 'content' in line.attrs:
			# print(line.attrs['content'])		
			id = line.attrs['content']

	if id:
		url ="https://www.ncbi.nlm.nih.gov/sviewer/viewer.fcgi?id={}&db=nuccore&report=genbank&conwithfeat=on&hide-cdd=on&retmode=text&ncbi_phid=CE88F25338A215A1000000000483042A&withmarkup=on&tool=portal&log$=seqview&maxdownloadsize=1000000".format(id)

	r2 = requests.get(url)
	locus += r2.content.decode('utf-8')
	print(locus)
	#utf-8 para converter de bytes para string
	#https://stackoverflow.com/questions/31019854/typeerror-cant-use-a-string-pattern-on-a-bytes-like-object-in-re-findall

Error: F a i l e d  t o  r e t r i e v e  s e q u e n c e :  9 6 0 6 



Error: F a i l e d  t o  r e t r i e v e  s e q u e n c e :  9 6 0 6 


Error: F a i l e d  t o  r e t r i e v e  s e q u e n c e :  3 4 8 



Error: F a i l e d  t o  r e t r i e v e  s e q u e n c e :  9 6 0 6 


Error: F a i l e d  t o  r e t r i e v e  s e q u e n c e :  3 4 8 


LOCUS       X54043                   707 bp    mRNA    linear   MAM 18-APR-2005
DEFINITION  Rabbit mRNA for myosin light chain 2, type 2.
ACCESSION   X54043
VERSION     X54043.1
KEYWORDS    fast skeletal muscle; myosin light chain.
SOURCE      Oryctolagus cuniculus (rabbit)
  ORGANISM  Oryctolagus cuniculus
            Eukaryota; Metazoa; Chordata; Craniata; Vertebrata; Euteleostomi;
            Mammalia; Eutheria; Euarchontoglires; Glires; Lagomorpha;
            Leporidae; Oryctolagus.
REFERENCE   1  (bases 1 to 610)
  AUTHORS   Maeda,K., Muller-Gerhardt,E. and Wittinghofer,A.
  TITLE     Sequence of two isoforms of myosin light cha

LINK NCBI

In [151]:
import re
id_lista = []
# print(genes)
existe = re.findall(r'ACCESSION\s+[^\s]+', locus)
#\s+ um ou mais espaços
#[^\s]+ indica um conjunto de caracteres que se podem repetir uma ou mais vezes

if existe:
    for c in existe:
        m = re.match(r'ACCESSION\s+([^\s]+)', c )
        if m:
            id = m.group(1)
            id_lista.append("https://www.ncbi.nlm.nih.gov/nuccore/{}".format( id ) )
else:
    print( "Padrão não encontrado" )
id_lista



['https://www.ncbi.nlm.nih.gov/nuccore/X54043',
 'https://www.ncbi.nlm.nih.gov/nuccore/X13699',
 'https://www.ncbi.nlm.nih.gov/nuccore/X56715',
 'https://www.ncbi.nlm.nih.gov/nuccore/X07052',
 'https://www.ncbi.nlm.nih.gov/nuccore/X05226']

ACCESSION

In [152]:
acc_list = []

existe = re.findall(r'ACCESSION\s+.*?(?=VERSION)', locus, re.DOTALL)
#\s+ um ou mais espaços
#\.* encontra o caractere ponto final zero ou mais vezes
#?= something que faz match com VERSION mas não consome estes carateres
if existe:
    for accession in existe:
        m = re.match( r'ACCESSION\s+(.+)', accession, re.DOTALL )
        acc_list.append(re.sub(r'\s+', ' ', m.group(1) ) )
#(.+) agrupar um ou mais caracteres dentro de um grupo
#re.sub(r'\s+', ' ' ...) substitui um ou mais espaços por apenas um espaço
acc_list

['X54043 ', 'X13699 ', 'X56715 ', 'X07052 ', 'X05226 ']

DEFINITION

In [153]:
defi_list = []
existe = re.findall(r'DEFINITION\s+.*?(?=ACCESSION)', locus, re.DOTALL)
if existe:
    for definition in existe:
        m = re.match( r'DEFINITION\s+(.+)', definition, re.DOTALL )
        defi_list.append(re.sub(r'\s+', ' ', m.group(1) ) )
defi_list

['Rabbit mRNA for myosin light chain 2, type 2. ',
 'Rabbit nonproductive VHa1 mRNA for Ig(mu) heavy chain constant region. ',
 'M. musculus rearranged T-cell receptor beta chain mRNA. ',
 'Horse zeta globin gene (BII allele). ',
 'Podospora anserina SU4 gene for suppressor tRNA-Ser (UGA). ']

LOCUS

In [154]:
loc_list = []
existe = re.findall(r'LOCUS\s+.*?(?=DEFINITION)', locus, re.DOTALL)
if existe:
    for loc in existe:
        m = re.match( r'LOCUS\s+(.+)', loc, re.DOTALL )
        loc_list.append(re.sub(r'\s+', ' ', m.group(1) ) )
loc_list

['X54043 707 bp mRNA linear MAM 18-APR-2005 ',
 'X13699 577 bp mRNA linear MAM 26-MAY-1992 ',
 'X56715 478 bp mRNA linear ROD 26-JUL-2016 ',
 'X07052 1563 bp DNA linear MAM 14-NOV-2006 ',
 'X05226 151 bp DNA linear PLN 06-JUL-1989 ']

LOCUS NAME

In [155]:
loc_name_list = []
existe = re.findall(r'LOCUS\s + [^\s]+', locus, re.DOTALL)
if existe:
    for id in existe:
        m = re.match( r'LOCUS\s+(.+)', id, re.DOTALL )
        loc_name_list.append(re.sub(r'\s+', ' ', m.group(1) ) )
loc_name_list

['X54043', 'X13699', 'X56715', 'X07052', 'X05226']

NUMBER OF BP

In [156]:
loc_bp_list = []
loc = []

existe = re.findall(r'LOCUS\s+.*?(?=DEFINITION)', locus, re.DOTALL)
# print(existe)
if existe:
    for c in existe:
        loc.append(list(filter(None, c.split(' '))))
for x in range(len(loc)):
    loc_bp_list.append(' '.join(loc[x][2:4])) 

#cria lista com os valores do locus, e escolhe os 3º e 4º caracteres, que correspondem ao nr de bp
loc_bp_list

['707 bp', '577 bp', '478 bp', '1563 bp', '151 bp']

MOLECULE TYPE

In [157]:
mol_type = []
for m in range(len(loc)):
    if "linear" in loc[m][5]:
        mol_type.append(' '.join(loc[m][4:6]))
    else:
        mol_type.append(' '.join(loc[m][4:5]))
#temos que corrigir esta porque as mol types podem ser de uma, duas ou trÊs palavras..
mol_type

['mRNA linear', 'mRNA linear', 'mRNA linear', 'DNA linear', 'DNA linear']

GENBANK DIVISION

In [158]:
gb_div = []
for gb in range(len(loc)):
    gb_div.append(' '.join(loc[gb][6:7]))
gb_div

['MAM', 'MAM', 'ROD', 'MAM', 'PLN']

MODIFICATION DATE

In [159]:
mod_date = []
for md in range(len(loc)):
    mod_date.append(''.join(loc[md][7]).replace("\n",""))
mod_date

['18-APR-2005', '26-MAY-1992', '26-JUL-2016', '14-NOV-2006', '06-JUL-1989']

SOURCE

In [160]:
source_list = []
existe = re.findall(r'SOURCE\s+.*?(?=ORGANISM)', locus, re.DOTALL)
# print(existe)
if existe:
    for source in existe:
        m = re.match( r'SOURCE\s+(.+)', source, re.DOTALL )
        source_list.append(re.sub(r'\s+', ' ', m.group(1) ) )
source_list

['Oryctolagus cuniculus (rabbit) ',
 'Oryctolagus cuniculus (rabbit) ',
 'Mus musculus (house mouse) ',
 'Equus caballus (horse) ',
 'Podospora anserina ']

ORGANISM

In [161]:
org_list = []
existe = re.findall(r'ORGANISM\s+.*?(?=\n)', locus, re.DOTALL)
if existe:
    for org in existe:
        m = re.match( r'ORGANISM\s+(.+)', org, re.DOTALL )
        org_list.append(re.sub(r'\s+', ' ', m.group(1) ) )
org_list

['Oryctolagus cuniculus',
 'Oryctolagus cuniculus',
 'Mus musculus',
 'Equus caballus',
 'Podospora anserina']

FEATURES

In [ ]:
misc = []
existe = re.findall(r'FEATURES\s+.*?(?=ORIGIN)', locus, re.DOTALL)
#DOTALL ignora as linhas, pois as features ocupam várias linhas
if existe:
    for feat in existe:
        m = re.match( r'FEATURES\s+(.+)',feat, re.DOTALL )
        misc.append(re.sub(r'/',  ' ', m.group(1) ) )
    for c in misc:
        print(c, "\n")

FEATURES - SOURCE

In [ ]:

existe = re.findall(r'source\s+[1-9\.]*', locus, re.DOTALL)
if existe:
    for fsource in existe:
        print( re.sub(r'source\s+', ' ', fsource) )


FEATURES - EXONS

In [ ]:
existe = re.findall(r'exon\s+[1-9\>\.]*', locus, re.DOTALL)
if existe:
    for fexons in existe:
        print( re.sub(r'exon\s+', ' ', fexons) )

FEATURES - INTRON

In [ ]:
existe = re.findall(r'intron\s+[1-9\>\.]*', locus, re.DOTALL)
if existe:
    for fintrons in existe:
        print( re.sub(r'intron\s+', ' ', fintrons) )

CDS - JOIN

In [ ]:
existe = re.findall(r'CDS\s+?(join)[1-9\.\,\(\)]*', locus, re.DOTALL)
#(join) procura a palavra join 1x
#procura nrs de 1-9, ponto, virgula e parentises zero ou mais vezes
print(existe[0])
if existe:
    for cds in existe:
        print( re.sub(r'CDS\s+', ' ', cds) )

SEQUENCE/ORIGIN

In [163]:
seq_origin = []
existe = re.findall(r'ORIGIN\s+.*?(?=//)', locus, re.DOTALL)
if existe:
    for nuc in existe:
        m = re.match( r'ORIGIN\s+(.+)', nuc, re.DOTALL)
        a = re.sub(r'\s+', '', m.group(1) ) 
        num = re.sub(r'\d+','', a)
        seq_origin.append(num)
seq_origin

['atggcacccaagaaggccaagagaagggcagcagcagagggcggcagctccaacgtcttctccatgttcgaccagactcagatccaggagttcaaggaggccttcaccgtcatcgatcagaaccgcgatggcatcatcgacaaggaagacctccgggacaccttcgcagccatgggccgcctcaatgtgaaaaatgaggagctggacgccatgatgaaggaagccagcggccccatcaacttcacggtcttcctgaccatgttcggggagaagctcaagggtgcggaccctgaggatgtcatcactggcgccttcaaagtgctggaccccgagggaaagggcacaatcaagaagcagttcttggaggagctgctgaccacgcagtgtgaccgcttctcccaggaggagatcaagaacatgtgggcggccttccccccggacgtgggcggcaacgtggactacaagaacatctgctacgtcatcacgcacggggacgccaaggaccaggagtaggcgagcgtctggcgcctccgctgacccgcgctcccggctagcccgagcccccgcttttttttttttttttttttttttttgttagaacaactcagcaaaataaaattccggtttattgttggacaacattgtttcacacatacatcaaacaggccaaaaaaaataaacagctaacttcatagacaaaaaaagga',
 'tagaaagctcaagcttagcttatcaatttctaaattgtattcatttttaaagtaagatattttagaaattaaacttacagatggtttattaatgcgcccatgaaactttttaaaattactttgttatttcttcatacggtaattaacctcaaacttcagagacctcaaagcattatttttgcaacgcctgcgaccgtcatttttaagagtctgttttctctatgattagaagagacgttttgaggcttcatttcagtatttttcaagactgcttttcaagtattca

In [164]:
seq_count = []
for seq in seq_origin:
    #contagem de nucleotidos
    A= []
    C= []
    T= []
    G= []
    for n in seq:
        if n == 'a':
            A.append('a')
        if n == 'c':
            C.append('c')
        if n == 't':
            T.append('t')
        if n == 'g':
            G.append('g')
        
    a = len(A)
    c = len(C)
    t = len(T)
    g = len(G)
    seq_count.append([a,c,t,g])
    #fazer append de um array
seq_count


[[190, 194, 136, 187],
 [145, 120, 180, 132],
 [121, 113, 127, 117],
 [299, 476, 274, 514],
 [41, 34, 44, 32]]

In [165]:
length = []
for seq in seq_origin:
    length.append(len(seq))
length

[707, 577, 478, 1563, 151]

JOURNAL

In [138]:
journ_list = []
existe = re.findall(r'JOURNAL\s+.*?(?=PUBMED|REFERENCE|FEATURES)', locus, re.DOTALL)
if existe:
    for journal in existe:
        m = re.match( r'JOURNAL\s+(.+)', journal, re.DOTALL )
        journ_list.append(re.sub(r'\s+', ' ', m.group(1) ) )
journ_list
#| dá-nos operador ou


['Proc. Natl. Acad. Sci. U.S.A. 91, 3690-3694 (1994) MEDLINE 94224807 ',
 'Submitted (15-NOV-1993) Geoffrey I. McFadden, School of Botany, University of Melbourne, Parkville, VIC 3052, Australia COMMENT [WARNING] On Jan 20, 1999 this sequence was replaced by U02075.1. ',
 'EMBO J. 7 (6), 1675-1682 (1988) MEDLINE 89005055 COMMENT [WARNING] On Feb 18, 1996 this sequence was replaced by X07874.1. see x07871 - x07874 Data kindly reviewed (12-SEP-1988) by CRUMPTON J. ',
 'Unpublished ',
 'Submitted (02-OCT-2002) Molecular Biology, PROIMI, Av. Belgrano y Pje. Caseros, S.M. de Tucuman, Tucuman 4000, Argentina ',
 'Unpublished (2002) COMMENT Contact: Nori Satoh Department of Zoology Kyoto University Sakyo-ku, Kyoto, Kyoto 606-8502, Japan. ']

AUTHORS

In [141]:
authors_list = []
existe = re.findall(r'AUTHORS\s+.*?(?=TITLE)', locus, re.DOTALL)
if existe:
    for auth in existe:
        m = re.match( r'AUTHORS\s+(.+)', auth, re.DOTALL )
        authors_list.append(re.sub(r'\s+', ' ', m.group(1) ) )
authors_list

['McFadden,G.I., Gilson,P.R., Hofmann,C.J., Adcock,G.J. and Maier,U.G. ',
 'McFadden,G.I. ',
 'Lang,G., Wotton,D., Owen,M.J., Sewell,W.A., Brown,M.H., Mason,D.Y., Crumpton,M.J. and Kioussis,D. ',
 'Benito,J.M., Lovrich,G.A., Abate,C.M. and Sineriz,F. ',
 'Benito,J.M., Lovrich,G.A., Abate,C.M. and Sineriz,F. ',
 'Satou,Y., Shin-i,T., Kohara,Y. and Satoh,N. ']

TITLE

In [142]:
titles = []
existe = re.findall(r'TITLE\s+.*?(?=JOURNAL)', locus, re.DOTALL)
if existe:
    for title in existe:
        m = re.match( r'TITLE\s+(.+)', title, re.DOTALL )
        titles.append(re.sub(r'\s+', ' ', m.group(1) ) )
titles

['Evidence that an amoeba acquired a chloroplast by retaining part of an engulfed eukaryotic alga ',
 'Direct Submission ',
 'The structure of the human CD2 gene and its expression in transgenic mice ',
 'Microbial populations from the subantarctic marine ecosystem ',
 'Direct Submission ',
 'Expressed genes in Ciona intestinalis (2002c) ']

PUBMED

In [ ]:
existe = re.findall(r'PUBMED\s+.*?(?=REFERENCE)', locus, re.DOTALL)
if existe:
    for pubm in existe:
        m = re.match( r'PUBMED\s+(.+)', pubm, re.DOTALL )
        print( re.sub(r'\s+', ' ', m.group(1) ) )

AFFILIATION

In [ ]:
import requests
r = requests.get('https://pubmed.ncbi.nlm.nih.gov/2549509')
pub = r.content.decode('utf-8')
print(pub)

In [ ]:
import re
existe = re.search(r'affiliation-1\s*.*\.', pub)
#\s* zero ou mais espaços
#.* zero ou mais caracteres
#\. caractere ponto final -> procura affiliation até encontrar ponto final

if existe:
    m = re.findall(r'>\s.*', existe[0] )
    #o findall retorna uma lista, por isso temos que selecionar m[0] para fazer o replace
    m = m[0].replace("> ",'')
    print(m)

else:
    print( "Padrão não encontrado" )

AUTHOR NAME

In [ ]:
existe = re.search("citation_authors.*;", pub)
if existe:
    m = re.findall(r'=".*', existe[0] )
    #procura =" seguido de um ou mais caracteres
    m = m[0].replace('=\"','')
    #\" para indicar que queremos os caractere " (e não abrir aspas)
    print(m)

CONNECTION TO DATABASE

In [166]:
import mysql.connector as SQLC
try: 
    DataBase = SQLC.connect(
    host ="127.0.0.1",
    user ="user",
    password ="password",
        database ="teste"
    )
    Cursor = DataBase.cursor()

    TableName ="""CREATE TABLE IF NOT EXISTS LOCUS
        (
        Id_locus VARCHAR(80) NOT NULL PRIMARY KEY,
        Sequence_length INT NULL,
        Mol_type VARCHAR(200) NULL,
        GB_division VARCHAR(30) NULL,
        Source VARCHAR(200) NULL,
        Organism VARCHAR(450) NULL,
        Accession VARCHAR(100) NOT NULL
        );
    """
    Cursor.execute(TableName)
    sql = "INSERT INTO LOCUS (Id_locus, Sequence_length, Mol_type, GB_division, Source, Organism, Accession) VALUES"
    for index in range(len(loc_name_list)):
        if index == len(loc_name_list)-1:
            sql += f" (\"{loc_name_list[index]}\", \"{length[index]}\", \"{mol_type[index]}\", \"{gb_div[index]}\", \"{source_list[index]}\", \"{org_list[index]}\", \"{acc_list[index]}\")"
        else:
            sql += f" (\"{loc_name_list[index]}\", \"{length[index]}\", \"{mol_type[index]}\", \"{gb_div[index]}\", \"{source_list[index]}\", \"{org_list[index]}\", \"{acc_list[index]}\"),"
    # print(sql)
    Cursor.execute(sql)

    DataBase.commit()

    print(Cursor.rowcount, "record inserted.")

    Cursor.close()

except SQLC.Error as error:
    print("Failed to insert record into MySQL table {}".format(error))

5 record inserted.
